In [585]:
# Import libraries
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

from numpy.polynomial import polynomial as P

In [586]:
# Times are in minutes
stations = [
    #Station Name,          East,   West
    ("Recto",               2.00,   7.00),
    ("Legarda",             2.75,   2.00),
    ("Pureza",              2.75,   2.75),
    ("V. Mapa",             2.50,   2.75),
    ("J. Ruiz",             1.75,   2.50),
    ("Gilmore",             2.25,   1.75),
    ("Betty-Go Belmonte",   2.25,   2.25),
    ("Araneta Cubao",       3.00,   2.25), 
    ("Anonas",              2.00,   3.00),
    ("Katipunan",           4.00,   2.00),
    ("Santolan",            3.50,   4.00),
    ("Marikina-Pasig",      4.50,   3.50),   
    ("Antipolo",            0.00,   4.50)
]

NSTATIONS = len(stations)



In [587]:
def read_data(filename):
    """Reads data from a file and returns a numpy array of the data"""
    data = pd.read_csv(filename, header=0)
    return data

monday_raw = read_data("Monday.csv")
monday_cum = monday_raw.cumsum()

monday_cum

,Recto:Entry,Recto:Exit,Legarda:Entry,Legarda:Exit,Pureza:Entry,Pureza:Exit,V. Mapa:Entry,V. Mapa:Exit,J. Ruiz:Entry,J. Ruiz:Exit,...,Anonas:Entry,Anonas:Exit,Katipunan:Entry,Katipunan:Exit,Santolan:Entry,Santolan:Exit,Marikina-Pasig:Entry,Marikina-Pasig:Exit,Antipolo:Entry,Antipolo:Exit
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,139.75,0.00,8.75,0.00,27.25,0.00,22.50,0.00,6.75,0.00,...,28.25,0.00,81.00,0.00,68.50,0.00,165.25,0.00,267.50,0.00
2,931.50,1339.50,108.75,570.50,243.50,227.75,226.25,167.75,87.75,95.50,...,409.00,212.75,807.75,145.00,530.50,105.00,1813.25,126.00,2760.00,113.00
3,2498.25,4405.75,341.50,3330.50,802.00,778.75,881.50,526.75,390.00,335.00,...,1180.50,1371.25,1697.50,780.75,1426.50,329.50,4681.50,488.75,7501.25,378.75
4,4480.50,8058.50,644.75,5537.25,1577.75,1448.25,1667.50,1150.25,669.50,773.00,...,1983.50,3154.75,2705.00,1598.75,2316.25,555.25,7387.50,995.75,11615.75,813.25
5,6122.50,11700.75,869.50,7056.50,2058.25,2238.00,2163.50,1784.00,863.75,1204.25,...,2601.25,4378.75,3742.50,2268.25,2952.50,725.75,9655.00,1495.25,14437.75,1214.00
6,7721.25,14263.75,1410.00,8156.50,2388.50,2944.00,2498.25,2291.25,1005.50,1396.25,...,3110.50,5136.50,4515.50,2726.25,3348.75,873.75,11187.50,1996.00,16519.75,1587.25
7,9274.50,16651.75,2151.50,9169.50,2757.25,3648.50,2811.25,2748.25,1108.00,1531.50,...,3655.25,5732.00,5102.00,3094.25,3641.25,1052.75,12325.50,2629.00,17970.00,2100.75
8,10972.50,18397.25,3032.75,9986.75,3188.25,4177.50,3137.00,3196.50,1216.75,1642.25,...,4221.50,6233.50,5685.00,3453.25,3893.00,1238.75,13421.50,3415.25,19385.50,2728.75
9,12705.00,20284.00,3965.75,11072.50,3706.50,4676.75,3548.50,3620.75,1320.75,1739.75,...,4843.25,6779.25,6302.00,3871.25,4171.00,1480.25,14479.00,4243.75,20584.50,3467.50


In [588]:
class Station():
    def __init__(self, name, entry_coeffs, exit_coeffs, idx, east_time, west_time):
        self.name = name
        # entry and exit in hours
        self.entry = P.Polynomial(entry_coeffs)
        self.exit = P.Polynomial(exit_coeffs)
        self.idx = idx
        self.east_time = east_time
        self.west_time = west_time

        # next stations
        self.east_next = None
        self.west_next = None

        # number of passengers waiting
        self.east_bound = 0
        self.west_bound = 0

        # number of potentially exiting passengers
        self.east_exiting = 0
        self.west_exiting = 0

    def step(self, start, step_size):
        # entering passengers
        entering = max(0, self.entry(start + step_size) - self.entry(start))

        self.east_bound += entering * (12 - self.idx) / 12
        self.west_bound += entering * self.idx / 12
        
        # potentially exiting passengers
        exiting = max(0, self.exit(start + step_size) - self.exit(start))

        self.east_exiting += exiting * self.idx / 12
        self.west_exiting += exiting * (12 - self.idx) / 12

        # print(self.name.ljust(20, ' '), end='')
        # print(", ".join([str(round(x, 2)).rjust(5, ' ') for x in [entering, self.east_bound, self.west_bound, self.east_exiting, self.west_exiting]]))

    def __str__(self):
        return self.name


class Depot():
    def __init__(self, name, headway, global_state, next_station):
        self.name = name
        self.headway = headway
        self.global_state = global_state
        self.next_station = next_station

        self.queue = []
        self.timer = 0
    
    def step(self, step_size):
        self.timer += step_size

        print("Depot timer:", self.timer * 60, "headway:", self.headway)
        print("Depot queue:", [x.name for x in self.queue])

        if self.timer * 60 >= self.headway - 0.001:
            if len(self.queue) <= 0:
                return

            train = self.queue.pop(0)
            train.next_station = self.next_station
            train.time_left = 0
            train.east = False

            # self.global_state.release_train(self.next_station, train.name, 0, False)
            self.timer = 0

    def reset_timer(self):
        self.timer = 0
    
    def enqueue(self, train):        
        self.queue.append(train)


class Train():
    def __init__(self, capacity, name, start_station, east, time_to_active):
        self.capacity = capacity
        self.name = name

        self.passengers = 0
        self.passenger_history = [(0, 0)]
        self.time_left = time_to_active
        self.east = east
        self.next_station = start_station

        # print("Train", self.name, "will start at", self.next_station.name, "in", round(self.time_left * 60, 2), "minutes")
    
    def step(self, time, step_size, headway, global_state):        
        self.time_left -= step_size

        print("Train", self.name, "en route to", self.next_station.name, "eastbound" if self.east else "westbound", "with", round(self.time_left * 60, 2), "minutes left")
        if self.time_left <= 0.0001:
            curr_station = self.next_station

            print("\nTrain", self.name, "arrived at", curr_station.name)
            Global.load_unload_passengers(time, self, curr_station, self.east)
            self.passenger_history.append((time, self.passengers))
            
            # reverse direction if in Recto
            if not self.east and curr_station == curr_station.west_next:
                self.east = True

            if not self.east and curr_station == global_state.stations[-1]:
                global_state.depot.reset_timer()

            if self.next_station == global_state.depot:
                global_state.depot.enqueue(self)


class Global():
    total_time = 2 * sum([x[2] for x in stations])

    def __init__(self, step_size, stations, ntrains):
        self.step_size = step_size
        self.stations = stations
        self.time = 0
        self.ids = list(range(1, 17))

        self.ntrains = ntrains
        self.headway = round(Global.total_time / ntrains * 4) / 4

        self.trains = []

        self.depot = Depot("Depot", self.headway, self, self.stations[-1])
        self.history = []


    def step(self, count = 1):
        for i in range(count):
            print("\nTime: ", round(self.time * 60, 2), "minutes")

            self.depot.step(self.step_size)

            for station in self.stations:
                station.step(self.time, self.step_size)
            
            max_passengers = 0

            for train in self.trains:
                if train.next_station == self.depot:
                    continue

                train.step(self.time, self.step_size, self.headway, self)

                if train.passengers > max_passengers:
                    max_passengers = train.passengers
            
            self.history.append((round(self.time * 60 * 4) / 4, max_passengers))

            self.time += self.step_size


    def release_train(self, station, name, time_to_active, east):
        train = Train(SEATING_CAP + STANDING_3CAP, name, station, east, time_to_active)
        self.trains.append(train)

        if station.name == "Depot":
            self.depot.enqueue(train)

    
    def add_train(self):
        self.ntrains += 1
        self.headway = round(Global.total_time / self.ntrains * 4) / 4
        
        self.depot.headway = self.headway

        # print(self.stations[-1].last_west, self.headway, self.time)
        # time_left = max(0, self.stations[-1].last_west + self.headway / 60 - self.time)

        print(f"New train will be released in depot")
        
        self.release_train(self.depot, self.ids.pop(0), 0, False)
    

    def remove_train(self):
        self.ntrains -= 1
        self.headway = round(Global.total_time / self.ntrains * 4) / 4

        self.depot.headway = self.headway


    @staticmethod
    def load_unload_passengers(time, train, station, east):
        print(station.name.ljust(20, ' '), end='')
        print(", ".join([str(round(x, 2)).rjust(5, ' ') for x in [station.east_bound, station.west_bound, station.east_exiting, station.west_exiting]]))
    
        if east:
            # Unload passengers
            exiting_passengers = max(0, math.floor(station.east_exiting))
            exiting_passengers = min(exiting_passengers, train.passengers)

            print("Unloading", exiting_passengers, "passengers")
            train.passengers -= exiting_passengers
            station.east_exiting -= exiting_passengers

            # load passengers
            loading_passengers = max(0, math.floor(station.east_bound))

            print("Loading", loading_passengers, "passengers")
            train.passengers += loading_passengers
            station.east_bound -= loading_passengers

            print("Train", train.name, "has", train.passengers, "passengers")

            train.time_left = station.east_time
            train.next_station = station.east_next
        else:
            # Set last west
            station.last_west = time
            
            # Unload passengers
            exiting_passengers = max(0, math.floor(station.west_exiting))
            exiting_passengers = min(exiting_passengers, train.passengers)

            print("Unloading", exiting_passengers, "passengers")
            train.passengers -= exiting_passengers
            station.west_exiting -= exiting_passengers

            # load passengers
            loading_passengers = max(0, math.floor(station.west_bound))

            print("Loading", loading_passengers, "passengers")
            train.passengers += loading_passengers
            station.west_bound -= loading_passengers

            print("Train", train.name, "has", train.passengers, "passengers")

            train.time_left = station.west_time
            train.next_station = station.west_next


In [589]:
SEATING_CAP = 232
STANDING_3CAP = 594
STANDING_4CAP = 793
STANDING_7CAP = 1396

interpolated = {}

for name, series in monday_cum.items():
    x = monday_cum.index.values
    y = series.values

    poly = P.polyfit(x, y, 10)

    interpolated[name] = poly

    
interpolated_df = pd.DataFrame(list(interpolated.items()), columns=['Name', 'Coefficients'])

interpolated_df

# P.Polynomial(interpolated_df[interpolated_df['Name'] == 'Recto:Entry']['Coefficients'].values[0])

,Name,Coefficients
0,Recto:Entry,"[4.093121663045758, 563.5378966795156, -1348.3..."
1,Recto:Exit,"[-4.111381440882949, 1009.2054196584809, -2635..."
2,Legarda:Entry,"[5.591467452368399, -173.883473528325, 131.949..."
3,Legarda:Exit,"[19.20754794087256, 873.9523147121585, -2740.9..."
4,Pureza:Entry,"[2.3290372498981182, 363.04891525364707, -823...."
5,Pureza:Exit,"[-2.0218619908236266, 240.57158545520866, -562..."
6,V. Mapa:Entry,"[2.593761381518774, 477.98371091551377, -1085...."
7,V. Mapa:Exit,"[-3.3566132416265537, 458.53700744984246, -902..."
8,J. Ruiz:Entry,"[1.6519144081496706, 151.48848591776914, -390...."
9,J. Ruiz:Exit,"[-2.8657101953978725, 412.57777496829704, -772..."


In [590]:
station_objs = []
train_objs = []

for station in stations:
    entry_poly = interpolated_df[interpolated_df['Name'] == station[0] + ':Entry']['Coefficients'].values[0]
    exit_poly = interpolated_df[interpolated_df['Name'] == station[0] + ':Exit']['Coefficients'].values[0]

    station_instance = Station(station[0], entry_poly, exit_poly, stations.index(station), station[1] / 60, station[2] / 60)

    station_objs.append(station_instance)

for i in range(len(station_objs) - 1):
    station_objs[i].east_next = station_objs[i + 1]
    station_objs[i + 1].west_next = station_objs[i]

station_objs[0].west_next = station_objs[0]

global_state = Global(0.25/60, station_objs, 8)
global_state.stations[-1].east_next = global_state.depot
global_state.depot.west_next = global_state.stations[-1]

half = math.floor(global_state.ntrains / 2)

for i in range(half):
    global_state.release_train(station_objs[0], global_state.ids.pop(0), (i * global_state.headway + 30.25) / 60, True)
    train_objs.append(global_state.trains[-1])

for i in range(half, global_state.ntrains):
    global_state.release_train(station_objs[-1], global_state.ids.pop(0), ((i - half) * global_state.headway + 30.25) / 60, False)
    train_objs.append(global_state.trains[-1])

# 4 to 6
global_state.step(2 * 60 * 4)

# 6 to 9
# global_state.add_train()
global_state.add_train()
global_state.step(3 * 60 * 4)

# 9 to 16
# global_state.remove_train()
global_state.remove_train()
global_state.step(7 * 60 * 4)

# 16 to 19
# global_state.add_train()
global_state.add_train()
global_state.step(3 * 60 * 4)

# 19 to 22
# global_state.remove_train()
global_state.remove_train()
global_state.step(3 * 60 * 4)

# interpolated_train_poly = P.polyfit(train_objs[0], train_0_count, 9)

print(global_state.history)

history = P.polyfit([x[0] for x in global_state.history], [x[1] for x in global_state.history], 10)

P.Polynomial(history)

# plt.plot([x[0] for x in global_state.history], [x[1] for x in global_state.history])
# plt.show()


# 137.77216166−20.41761589x+0.65782308x^2−0.00609801x^3+(2.90592135*10^{-05})x^4−(8.32579123*10^{-08})x^5+(1.51494375*10^{-10})x^6−(1.76372821*10^{-13})x^7+(1.27270759*10^{-16})x^8−(5.18496315*10^{-20})x^9+(9.1169026*10^{-24})x^{10}
# 113.92309974−16.46376726x+0.51557635x^2−0.00416856x^3+(1.66790986*10^{-05})x^4−(3.92097016*10^{-08})x^5+(5.76222779*10^{-11})x^6−(5.35134735*10^{-14})x^7+(3.04881584*10^{-17})x^8−(9.73778361*10^{-21})x^9+(1.3411779*10^{-24})x^{10}
# 106.90875466−14.37670454x+0.4210205x^2−0.00273611x^3+(7.02078031*10^{-06})x^4−(3.93449543*10^{-09})x^5−(1.87380712*10^{-11})x^6+(4.74502008*10^{-14})x^7−(4.96074673*10^{-17})x^8+(2.52889191*10^{-20})x^9−(5.15007845*10^{-24})x^{10}


Time:  0 minutes
Depot timer: 0.25 headway: 10.0
Depot queue: []
Train 1 en route to Recto eastbound with 30.0 minutes left
Train 2 en route to Recto eastbound with 40.0 minutes left
Train 3 en route to Recto eastbound with 50.0 minutes left
Train 4 en route to Recto eastbound with 60.0 minutes left
Train 5 en route to Antipolo westbound with 30.0 minutes left
Train 6 en route to Antipolo westbound with 40.0 minutes left
Train 7 en route to Antipolo westbound with 50.0 minutes left
Train 8 en route to Antipolo westbound with 60.0 minutes left

Time:  0.25 minutes
Depot timer: 0.5 headway: 10.0
Depot queue: []
Train 1 en route to Recto eastbound with 29.75 minutes left
Train 2 en route to Recto eastbound with 39.75 minutes left
Train 3 en route to Recto eastbound with 49.75 minutes left
Train 4 en route to Recto eastbound with 59.75 minutes left
Train 5 en route to Antipolo westbound with 29.75 minutes left
Train 6 en route to Antipolo westbound with 39.75 minutes left
Train 7 en route

Polynomial([ 1.13923100e+02, -1.64637673e+01,  5.15576354e-01, -4.16855733e-03,
        1.66790986e-05, -3.92097016e-08,  5.76222779e-11, -5.35134735e-14,
        3.04881584e-17, -9.73778361e-21,  1.34117790e-24], domain=[-1,  1], window=[-1,  1], symbol='x')

# TODO
- Figure out how to change NTRAINS dynamically and adjust spacing between trains
- Figure out how to handle initially odd NTRAINS
- Train and station capacity computations